<a href="https://colab.research.google.com/github/LordHandLee/HonCSC499_detect_falls_soccer/blob/main/Copy_of_ReconstructedModel_02_6_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
pip install autokeras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.4/415.4 kB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 68.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 92.5 MB/s eta 0:00:00


In [9]:
import autokeras as ak
import numpy as np
import tensorflow as tf
from tensorflow import keras
import glob

import matplotlib.pyplot as plt
import cv2
from skimage import data,io
import matplotlib.image as mpimg

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
from tensorflow.keras.layers.experimental import preprocessing
from autokeras.utils import data_utils

In [2]:
base_model = keras.applications.efficientnet.EfficientNetB1(
    include_top=False,
    weights='imagenet',
    input_shape=(224,224,3),
)


27018416/27018416 [==============================] - 2s 0us/step


In [10]:


#




@keras.utils.register_keras_serializable()
class CastToFloat32(preprocessing.PreprocessingLayer):
    def get_config(self):
        return super().get_config()

    def call(self, inputs):
        return data_utils.cast_to_float32(inputs)

    def adapt(self, data):
        return

In [4]:
base_model.trainable = True

In [13]:
inputs = keras.Input(shape=(150,150,3))
#x = base_model(inputs, training=False)

#x = tf.cast(inputs, tf.int32)
x = CastToFloat32()(inputs)

#x = CastToFloat32(inputs, shape=(None, 150, 150, 3))

x = keras.layers.Normalization(axis=-1, mean=None, variance=None, invert=False)(x)

x = keras.layers.RandomTranslation(height_factor=0.1, width_factor=0.1, fill_mode='reflect', fill_value=0.0, interpolation='bilinear', seed=None)(x)

x = keras.layers.RandomFlip(mode='horizontal', seed=None)(x)

x = keras.layers.Resizing(height=224, width=224, interpolation='bilinear', crop_to_aspect_ratio=False)(x)

x = base_model(x, training=True)

x = keras.layers.GlobalAveragePooling2D(data_format='channels_last', keepdims=False)(x)

x = keras.layers.Dense(1, activation='linear', use_bias=True, kernel_initializer='GlorotUniform', bias_initializer='zeros', kernel_regularizer= None, bias_regularizer= None, activity_regularizer= None, kernel_constraint= None, bias_constraint= None)(x)


#dropout?

outputs = keras.layers.Activation('sigmoid')(x)

model = keras.Model(inputs, outputs)




#  random_flip (RandomFlip)    (None, 150, 150, 3)       0

#  resizing (Resizing)         (None, 224, 224, 3)       0

#  efficientnetb1 (Functional  (None, None, None, 1280   6575239
#  )                           )

#  global_average_pooling2d (  (None, 1280)              0
#  GlobalAveragePooling2D)

#  dense (Dense)               (None, 1)                 1281

#  classification_head_1 (Ac



In [14]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 cast_to_float32_1 (CastToF  (None, 150, 150, 3)       0         
 loat32)                                                         
                                                                 
 normalization_2 (Normaliza  (None, 150, 150, 3)       7         
 tion)                                                           
                                                                 
 random_translation_1 (Rand  (None, 150, 150, 3)       0         
 omTranslation)                                                  
                                                                 
 random_flip_1 (RandomFlip)  (None, 150, 150, 3)       0         
                                                             

In [15]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01),
              loss=keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=[keras.metrics.BinaryAccuracy()])

In [16]:
early_stop = EarlyStopping(monitor='val_loss', min_delta=0.00001, patience=30)

In [17]:
from google.colab import drive

In [18]:
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [19]:
data_dir = "/content/drive/MyDrive/honor_thesis/image_depo/"

In [20]:
datagen = ImageDataGenerator(rescale=1/255.0, validation_split=0.3)
width = 150
height = 150
trainDatagen = datagen.flow_from_directory(directory=data_dir, target_size=(width, height),
                                           class_mode='binary',
                                           batch_size=16,
                                           subset='training')
valDatagen = datagen.flow_from_directory(directory=data_dir, target_size=(width, height),
                                           class_mode='binary',
                                           batch_size=16,
                                           subset='validation')

Found 611 images belonging to 2 classes.
Found 260 images belonging to 2 classes.


In [21]:
ls "/content/drive/MyDrive/honor_thesis/image_depo/"

fallen_images/  model_autokeras.h5  model_autokeras.keras  standing_images/


In [22]:
history = model.fit(trainDatagen, steps_per_epoch = len(trainDatagen),
                    epochs=500, validation_data = valDatagen,
                    validation_steps = len(valDatagen),
                    callbacks=[early_stop])

Epoch 1/500
39/39 [==============================] - 392s 9s/step - loss: 0.9024 - binary_accuracy: 0.5974 - val_loss: 1.1385 - val_binary_accuracy: 0.5769
Epoch 2/500
39/39 [==============================] - 4s 104ms/step - loss: 0.7670 - binary_accuracy: 0.5957 - val_loss: 0.6855 - val_binary_accuracy: 0.7077
Epoch 3/500
39/39 [==============================] - 4s 105ms/step - loss: 0.4442 - binary_accuracy: 0.7971 - val_loss: 0.5780 - val_binary_accuracy: 0.6615
Epoch 4/500
39/39 [==============================] - 4s 102ms/step - loss: 0.5861 - binary_accuracy: 0.7152 - val_loss: 0.4276 - val_binary_accuracy: 0.8192
Epoch 5/500
39/39 [==============================] - 4s 103ms/step - loss: 0.3057 - binary_accuracy: 0.8887 - val_loss: 0.3759 - val_binary_accuracy: 0.8423
Epoch 6/500
39/39 [==============================] - 4s 104ms/step - loss: 0.2562 - binary_accuracy: 0.9067 - val_loss: 0.2821 - val_binary_accuracy: 0.8962
Epoch 7/500
39/39 [==============================] - 4s 103

In [ ]:
history = model.fit(trainDatagen, steps_per_epoch = len(trainDatagen),
                    epochs=500, validation_data = valDatagen,
                    validation_steps = len(valDatagen),
                    callbacks=[early_stop])

Epoch 1/500
39/39 [==============================] - 63s 152ms/step - loss: 0.9531 - binary_accuracy: 0.5123 - val_loss: 0.7326 - val_binary_accuracy: 0.5000
Epoch 2/500
39/39 [==============================] - 4s 89ms/step - loss: 0.7088 - binary_accuracy: 0.5548 - val_loss: 0.7354 - val_binary_accuracy: 0.4962
Epoch 3/500
39/39 [==============================] - 4s 90ms/step - loss: 0.7076 - binary_accuracy: 0.5417 - val_loss: 0.7012 - val_binary_accuracy: 0.5038
Epoch 4/500
39/39 [==============================] - 3s 87ms/step - loss: 0.7132 - binary_accuracy: 0.5106 - val_loss: 0.6888 - val_binary_accuracy: 0.5462
Epoch 5/500
39/39 [==============================] - 3s 88ms/step - loss: 0.6692 - binary_accuracy: 0.5679 - val_loss: 0.7334 - val_binary_accuracy: 0.5538
Epoch 6/500
39/39 [==============================] - 3s 89ms/step - loss: 0.6771 - binary_accuracy: 0.5646 - val_loss: 0.6916 - val_binary_accuracy: 0.5538
Epoch 7/500
39/39 [==============================] - 4s 89ms/s